# Setting Up Environment

In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select
from webdriver_manager.chrome import ChromeDriverManager

import time
import pandas as pd
import re
from fuzzywuzzy import fuzz
from bs4 import BeautifulSoup
import datetime as dt
import numpy as np

/opt/anaconda3/lib/python3.9/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


# Fixing Odds Scraper

In [3]:
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get('https://www.actionnetwork.com/ufc/odds')

[WDM] - Downloading: 100%|█████████████████| 8.29M/8.29M [00:00<00:00, 20.3MB/s]
/var/folders/xz/nv9k4jpn7l9dtk4l7yqbl8640000gn/T/ipykernel_33284/768927109.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [4]:
html = driver.page_source
tables = pd.read_html(html)
odds = tables[0]
odds = odds.iloc[::2]
odds.reset_index(drop = True, inplace = True)

In [5]:
odds

,Scheduled,Open,Best Odds,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11
0,Esteban RibovicsE. RibovicsKamuela KirkK. Kirk,-200+170,-149+135,-149+125,-155+125,-150+125,N/AN/A,-155+135,-152+118,-150+120,N/AN/A,-150+125
1,Jesus AguilarJ. AguilarShannon RossS. Ross,-230+195,-140+125,-145+120,-140+115,-140+120,N/AN/A,-145+125,-143+114,-143+116,N/AN/A,-150+125
2,Terrence MitchellT. MitchellCameron SaaimanC. ...,+350-450,+430-550,+425-600,+400-550,+400-550,N/AN/A,+430-600,+380-560,+400-560,N/AN/A,+400-550
3,🇵🇱Marcin PrachnioM. PrachnioVitor PetrinoV. Pe...,+210-250,+200-230,+200-250,+190-250,+200-240,N/AN/A,+195-230,+188-245,+195-245,N/AN/A,+200-250
4,Edgar ChairezE. ChairezTatsuro TairaT. Taira,+600-910,+675-975,+625-1000,+675-1100,+650-1000,N/AN/A,+675-975,N/AN/A,+650-1000,N/AN/A,+600-1000
5,🇺🇸Alonzo MenifieldA. MenifieldJim CruteJ. Crute,-120+100,+120-130,+110-135,+105-130,+115-135,N/AN/A,+120-140,+116-150,+118-148,N/AN/A,+110-138
6,Denise GomesD. GomesYazmin JaureguiY. Jauregui,+370-460,+300-369,+290-369,+290-375,+300-380,N/AN/A,+300-400,+275-375,+285-375,N/AN/A,+300-400
7,Jack Della MaddalenaJ. MaddalenaJosiah Harrell...,-1115+650,-900+675,-1000+625,-1000+625,-900+600,N/AN/A,-1100+675,-1000+600,-1000+650,N/AN/A,-1000+600
8,🇺🇸Niko PriceN. Price🇺🇸Robbie LawlerR. Lawler,-240+200,-225+200,-230+185,-250+195,-225+185,N/AN/A,-225+190,-235+180,-235+185,N/AN/A,-250+200
9,BO NickalB. NickalVal WoodburnV. Woodburn,-2500+1100,-1800+1300,-2500+1000,-2500+1100,-2800+1300,N/AN/A,-1800+1000,-3335+1150,-3335+1300,N/AN/A,-2500+1100
